# Project: Apply Lightweight Fine-Tuning to a Foundation Model

## Prepare the Foundation Model


### Imports

In [1]:
import torch
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import datasets
import numpy as np


/opt/homebrew/Caskroom/miniconda/base/envs/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Dataset and compute functions

In [2]:
def get_dataset_split(name: str, name_split: str):
    dataset = datasets.load_dataset(name,  name_split, split='train').train_test_split(
        test_size=0.2, shuffle=True, seed=23

    )
    return dataset['train'], dataset['test']


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

def predict(text: str, model, tokenizer):
    input = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        output = model(**input)

    logits = output.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predicted_class_idx = torch.argmax(probabilities, dim=-1).item()
    return predicted_class_idx


In [3]:
BASE_MODEL = "gpt2"
label2id = {"neutral": 1, "positive": 2, "negative": 0}
id2label = {1: "neutral", 2: "positive", 0: "negative"}


### Load a pretrained HF model

In [4]:
base_model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL,
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Load and preprocess a dataset

In [5]:
train, test = get_dataset_split('financial_phrasebank', 'sentences_66agree')

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

def tokenize(batch):
    return tokenizer(batch['sentence'], return_tensors="pt", truncation=True, max_length=512, padding=True)

### Evaluate the pretrained model

In [6]:
test_df = pd.DataFrame(test)
test_small = test_df.sample(100)
test_small['predicted'] = test_small['sentence'].apply(lambda x: predict(x, base_model, tokenizer))
test_small['correct'] = test_small['predicted'] == test_small['label']
prediction_percentage = test_small['correct'].mean()
print(f"Accuracy: {prediction_percentage:.2f}")


Accuracy: 0.11


## Perform Lightweight Fine-Tuning

### Create a PEFT model

In [7]:
from peft import LoraConfig, get_peft_model
from peft import AutoPeftModelForCausalLM

config = LoraConfig()
number_labels = len(list(id2label.keys()))
model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL,
    num_labels=number_labels,
    id2label=id2label,
    label2id=label2id
)
lora_model = get_peft_model(model, config)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<pad>'})

lora_model.config.pad_token_id = tokenizer.pad_token_id


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/homebrew/Caskroom/miniconda/base/envs/nlp/lib/python3.12/site-packages/peft/tuners/lora/layer.py:711: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


### Train the PEFT model

In [9]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

# We need to remove the sentence column from the datasets. Because there is a bug in the current version of the library
train_peft = train.map(tokenize, batched=True).remove_columns(["sentence"])
test_peft = test.map(tokenize, batched=True).remove_columns(["sentence"])

# Rename the label column to labels because the trainer expects that name
train_peft = train_peft.rename_column("label", "labels")
test_peft = test_peft.rename_column("label", "labels")

training_args = TrainingArguments(
    output_dir="./data/financial_phrasebank",
    num_train_epochs=10,
    learning_rate=2e-3,

    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,

    weight_decay=0.01,

    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    remove_unused_columns=False,
    label_names=["labels"],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_peft,
    eval_dataset=test_peft,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer)
)
trainer.train()

Map: 100%|██████████| 844/844 [00:00<00:00, 13961.99 examples/s]










































                                        

                                         
  0%|          | 0/2110 [01:20<?, ?it/s]          

Checkpoint destination directory ./data/financial_phrasebank/checkpoint-211 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'eval_loss': 0.3198302090167999, 'eval_accuracy': 0.8625592417061612, 'eval_runtime': 5.4967, 'eval_samples_per_second': 153.547, 'eval_steps_per_second': 9.642, 'epoch': 1.0}





























                                        

                                         
  0%|          | 0/2110 [02:27<?, ?it/s]          

Checkpoint destination directory ./data/financial_phrasebank/checkpoint-422 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'eval_loss': 0.2886766195297241, 'eval_accuracy': 0.8862559241706162, 'eval_runtime': 5.2185, 'eval_samples_per_second': 161.731, 'eval_steps_per_second': 10.156, 'epoch': 2.0}


                                        
  0%|          | 0/2110 [02:50<?, ?it/s]          

...




























                                        

                                         
  0%|          | 0/2110 [11:31<?, ?it/s]           

Checkpoint destination directory ./data/financial_phrasebank/checkpoint-2110 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'eval_loss': 0.4792298674583435, 'eval_accuracy': 0.8886255924170616, 'eval_runtime': 5.2082, 'eval_samples_per_second': 162.052, 'eval_steps_per_second': 10.176, 'epoch': 10.0}


                                        
100%|██████████| 2110/2110 [11:15<00:00,  3.12it/s]

{'train_runtime': 675.5911, 'train_samples_per_second': 49.927, 'train_steps_per_second': 3.123, 'train_loss': 0.1917101031796062, 'epoch': 10.0}


TrainOutput(global_step=2110, training_loss=0.1917101031796062, metrics={'train_runtime': 675.5911, 'train_samples_per_second': 49.927, 'train_steps_per_second': 3.123, 'train_loss': 0.1917101031796062, 'epoch': 10.0})

### Save the PEFT model

In [10]:
lora_model.save_pretrained('gpt-lora')

## Perform Inference Using the Fine-Tuned Model


### Load the saved PEFT model

In [11]:
lora_model_finetuned = AutoPeftModelForCausalLM.from_pretrained('gpt-lora')


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Evaluate the fine-tuned model

In [14]:
test_df_finetuned = pd.DataFrame(test)
test_small_finetuned = test_df_finetuned.sample(100)
test_small_finetuned['predicted'] = test_small_finetuned['sentence'].apply(lambda x: predict(x, lora_model_finetuned, tokenizer))
test_small_finetuned['correct'] = test_small_finetuned['predicted'] == test_small_finetuned['label']
prediction_percentage_finetuned_peft = test_small_finetuned['correct'].mean()
print(f"Accuracy: {prediction_percentage_finetuned_peft:.2f}")

Accuracy: 0.42


## Comparisons

In [17]:
print(f"Accuracy base model: {prediction_percentage:.2f}")
print(f"Accuracy peft model: {prediction_percentage_finetuned_peft:.2f}", f"Improvement: {prediction_percentage_finetuned_peft - prediction_percentage:.2f}")

Accuracy base model: 0.14
Accuracy peft model: 0.42 Improvement: 0.28
